# Example of how to use the Orca wrapper


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
import logging

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import MolsToGridImage, MolToImage
from rdkit.Chem.MolStandardize import rdMolStandardize

In [ ]:
try:
    import ppqm
except ModuleNotFoundError:
    import pathlib

    cwd = pathlib.Path().resolve().parent
    sys.path.append(str(cwd))
    import ppqm

In [ ]:
from ppqm import orca
from ppqm import jupyter as ppqm_jupyter

## Set logging level

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("orca").setLevel(logging.DEBUG)
show_progress = True

In [ ]:
_logger = logging.getLogger(__name__)

## Define a molecule you like

In [ ]:
smiles = "C([C@@H](C(=O)O)N)O"  # I like serine
molobj = Chem.MolFromSmiles(smiles)

In [ ]:
molobj

## Define an Orca calculator object

In [ ]:
orca_options = {
    "scr": "./_tmp_directory_",  # Where should the calculations happen?
    "cmd": "orca",  # Where is the binary executable/command?
    "n_cores": 8,  # How many cores to use?
    "show_progress": True,  # Show progressbar during calculation
}

In [ ]:
calc = orca.OrcaCalculator(**orca_options)
calc

## Define what you would like to calculate

In [ ]:
# Calculate values for molecule in water
calculation_option = {
    "NumFreq": None,
    "B3LYP": None,
    "def2-SVP": None,
    "D3BJ": None,
    "Hirshfeld": None,
    "CPCM": "water",
    "RIJCOSX": None,
    "def2/J": None,
    "Grid4": None,
    "GridX4": None,
    "NMR": None,
    "def2/JK": None
}

In [ ]:
# Generate conformers
molobj_conf = ppqm.tasks.generate_conformers(molobj, max_conformers=3)

In [ ]:
molobj_conf.GetNumConformers()

In [ ]:
# Calculate energy of conformers
results = calc.calculate(molobj_conf, calculation_option)

In [ ]:
# The output type
print(type(results))
print(type(results[0]))
print(results[0].keys())

In [ ]:
# Collect all the orca energies
conformer_energies = [result["scf_energy"] for result in results]

In [ ]:
# Find the lowest energy
idx = np.argmin(conformer_energies)
idx

## Transfer the results into a Pandas dataframe

In [ ]:
df = pd.DataFrame(results)
df

In [ ]:
# get coordinates for lowest energy conformer
coord = results[idx]["coord"]

In [ ]:
# get mulliken charges for lowest energy conformer
mulliken = results[idx]["mulliken_charges"]
mulliken

In [ ]:
# Copy only the graph of the molecule, keeping the atom order
molobj_prime = ppqm.chembridge.copy_molobj(molobj_conf)

# Get the coordinates of the best conformer
coord = results[idx]["coord"]

# Add conformer to molobj
ppqm.chembridge.molobj_add_conformer(molobj_prime, coord)

In [ ]:
ppqm_jupyter.show_molobj(molobj_prime)

In [ ]:
for coef, atom in zip(mulliken, molobj_prime.GetAtoms()):
    label = f"{coef:.2f}"
    atom.SetProp("atomNote", label)

In [ ]:
MolToImage(
    Chem.RemoveHs(molobj_prime),
    size=(300, 300),
)